In [1]:
# get tasks from json file
import json
with open('./task_1_plan_generation.json', 'r') as f:
    tasks_json = f.read()
complete_tasks_obj = json.loads(tasks_json)
task_objs = complete_tasks_obj['instances']

FileNotFoundError: [Errno 2] No such file or directory: './task_1_plan_generation.json'

In [ ]:
from dataclasses import dataclass

@dataclass
class Task:
    id: str
    query: str
    solution: str

In [ ]:
tasks = []
for task_obj in task_objs:
    task = Task(
        id=task_obj['instance_id'],
        query=task_obj['query'],
        solution=task_obj['ground_truth_plan']
    )
    tasks.append(task)

In [2]:
print(f"Loaded {len(tasks)} tasks from JSON file.")
print("Example task:")
print(tasks[0])

NameError: name 'tasks' is not defined

In [13]:
from tot import TreeOfThoughts

@dataclass
class Result:
    task_id: str
    success: bool
    answer: str

async def run_task(tot: TreeOfThoughts, task: Task) -> Result:
    # Run the task using the Tree of Thoughts framework
    success, answer = await tot.run_against(task.query, task.solution)
    return Result(
        task_id=task.id,
        success=success,
        answer=answer
    )

async def run_tasks(tot: TreeOfThoughts, tasks: list[Task]) -> list[Result]:
    results = []
    for task in tasks:
        result = await run_task(tot, task)
        results.append(result)
    return results

In [ ]:
from llm_engine import Engine
from dotenv import load_dotenv
import os
from openai import AsyncOpenAI
from tot_config import Configuration

mode = "openai"

if mode == "openai":
    # OpenAI client
    load_dotenv(".env")

    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

    client_openai = AsyncOpenAI(api_key=OPENAI_API_KEY)

    engine = Engine(client_openai, chat=True, model="gpt-4o-mini")

elif mode == "vllm":
    NGROK_LINK = "https://" + "fitting-shrimp-pleasantly.ngrok-free.app"

    client_vllm = AsyncOpenAI(
        api_key="None",
        base_url=str(NGROK_LINK + "/v1")
    )

    engine = Engine(client_vllm, chat=True, model="Qwen/Qwen2.5-32B-Instruct-GPTQ-Int8")

else:
    raise ValueError("Invalid mode. Choose 'openai' or 'vllm'")

tot = TreeOfThoughts(
    llm_engine=engine,
    goal_verifier=None,
    max_depth=1,
    branch_factor=2,
    config=Configuration.from_file("base")
)


In [15]:
#tasks[0].query = "I am playing with a set of blocks where I need to arrange the blocks into stacks. Here are the actions I can do\n\nPick up a block\nUnstack a block from on top of another block\nPut down a block\nStack a block on top of another block\n\nI have the following restrictions on my actions:\nI can only pick up or unstack one block at a time.\nI can only pick up or unstack a block if my hand is empty.\nI can only pick up a block if the block is on the table and the block is clear. A block is clear if the block has no other blocks on top of it and if the block is not picked up.\nI can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block.\nI can only unstack a block from on top of another block if the block I am unstacking is clear.\nOnce I pick up or unstack a block, I am holding the block.\nI can only put down a block that I am holding.\nI can only stack a block on top of another block if I am holding the block being stacked.\nI can only stack a block on top of another block if the block onto which I am stacking the block is clear.\nOnce I put down or stack a block, my hand becomes empty.\nOnce you stack a block on top of a second block, the second block is no longer clear.\n\n[STATEMENT]\nAs initial conditions I have that, the red block is clear, the blue block is clear, the yellow block is clear, the hand is empty, the blue block is on top of the orange block, the red block is on the table, the orange block is on the table and the yellow block is on the table.\nMy goal is to have that the orange block is on top of the blue block.\n\nMy plan is as follows:\n\n[PLAN]\nunstack the blue block from on top of the orange block\nput down the blue block\npick up the orange block\nstack the orange block on top of the blue block\n[PLAN END]\n\n[STATEMENT]\nAs initial conditions I have that, the red block is clear, the yellow block is clear, the hand is empty, the red block is on top of the blue block, the yellow block is on top of the orange block, the blue block is on the table and the orange block is on the table.\nMy goal is to have that the orange block is on top of the red block.\n\nMy plan is as follows:\n\n[PLAN]"
tasks[0].query = "I am playing with a set of blocks where I need to arrange the blocks into stacks. Here are the actions I can do\n\nPick up a block\nUnstack a block from on top of another block\nPut down a block\nStack a block on top of another block\n\nI have the following restrictions on my actions:\nI can only pick up or unstack one block at a time.\nI can only pick up or unstack a block if my hand is empty.\nI can only pick up a block if the block is on the table and the block is clear. A block is clear if the block has no other blocks on top of it and if the block is not picked up.\nI can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block.\nI can only unstack a block from on top of another block if the block I am unstacking is clear.\nOnce I pick up or unstack a block, I am holding the block.\nI can only put down a block that I am holding.\nI can only stack a block on top of another block if I am holding the block being stacked.\nI can only stack a block on top of another block if the block onto which I am stacking the block is clear.\nOnce I put down or stack a block, my hand becomes empty.\nOnce you stack a block on top of a second block, the second block is no longer clear.\n\n[STATEMENT]\nAs initial conditions I have that, the red block is clear, the yellow block is clear, the hand is empty, the red block is on top of the blue block, the yellow block is on top of the orange block, the blue block is on the table and the orange block is on the table.\nMy goal is to have that the orange block is on top of the red block.\n\nMy plan is as follows:\n\n[PLAN]"

# set logging to info
import logging
import sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], force=True)

results = await run_task(tot, tasks[0])

print(f"Task ID: {results.task_id}")
print(f"Success: {results.success}")
print(f"Answer: {results.answer}")

INFO:root:Starting 2 parallel LLM requests at depth 1 (parent uuid: 9d0f8969ae3b4f43b3ba82f098e4291b)
INFO:root:Processing branch at depth 1 (parent uuid: 9d0f8969ae3b4f43b3ba82f098e4291b)
INFO:root:Processing branch at depth 1 (parent uuid: 9d0f8969ae3b4f43b3ba82f098e4291b)
INFO:httpx:HTTP Request: POST https://fitting-shrimp-pleasantly.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://fitting-shrimp-pleasantly.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Generated successor at depth 1 (parent uuid: 9d0f8969ae3b4f43b3ba82f098e4291b)
INFO:httpx:HTTP Request: POST https://fitting-shrimp-pleasantly.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://fitting-shrimp-pleasantly.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Generated successor at depth 1 (parent uuid: 9d0f8969ae3b4f43b3ba82f098e4291b)
INFO:root:Finished 2 parallel LLM requests at depth 1 (parent uuid: 9d0f8969ae

In [8]:
# save tree to file as json
import json
with open("tot.json", "w") as f:
    f.write(tot.serialize())
with open("tree.json", "w") as f:
    f.write(json.dumps(tot.init_state.to_dict(), indent=4))

In [9]:
from tot import TreeOfThoughts
import json
# load tree from file as json
with open("tree.json", "r") as f:
    tree = json.load(f)
    tree = TreeOfThoughts.State.from_dict(tree)

In [12]:
from tree_gui import get_tree_view
from PySide6.QtWidgets import QApplication
import qdarkstyle

# Create a tree view using the TreeOfThoughts class
app = QApplication.instance()
if not app:
    app = QApplication([])
app.setStyleSheet(qdarkstyle.load_stylesheet(qt_api='pyside6'))
tree_view = get_tree_view(tree, 80, 150)
tree_view.show()
app.exec()

INFO:qdarkstyle:QSS file successfully loaded.
INFO:qdarkstyle:Found application patches to be applied.


0

In [1]:
from tot_config import Configuration

base_config = Configuration(
    succ_prompt="You are a smart assistant that answers concisely.\nYou will be given a problem / question to solve.\nYou must output exactly the next step in solving the problem when prompted with [NEXT]. Output [FINISHED] when you are finished with solving the problem.\nThis is your problem statement:",
    gen_succ_prompt="[NEXT]\n",
)

stream_config = Configuration(
    stream_prompt="You are a smart assistant that answers concisely.\nYou will be given a problem / question to solve.\nOutput [FINISHED] when you are done. This is your problem statement:",
)

base_config.to_file("base")

stream_config.to_file("stream")
